## map2loop: Topology fom Local file or Loop WFS Server
This notebook reads in two layers from the local files:  geology polygons and fault polylines; and calculates the topological relationships between the different features. Requires compiled cpp code from Vitaliy Ogarko

In [6]:
import subprocess
import geopandas as gpd
import os
import sys
import stat
import functools 
import operator  
import matplotlib
import networkx as nx
from shapely.geometry import Polygon
from map2loop import m2l_utils
%matplotlib inline

newwd="C:\\Users\\00073294\Dropbox\\loop_minex\\map2model\\"
#os.chdir(newwd)
print("Current Working Directory " )

gdal_data = os.environ['GDAL_DATA']
print("***",gdal_data)
print('is dir: ' + str(os.path.isdir(gdal_data)))
gcs_csv = os.path.join(gdal_data, 'gcs.csv')
print('is file: ' + str(os.path.isfile(gcs_csv)))
st = os.stat(gcs_csv)
print('is readable: ' + str(bool(st.st_mode & stat.S_IRGRP)))
os.environ['PROJ_LIB']=r"C:\\Users\\00073294\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\pyproj\\proj_dir\\share\\proj"
print(os.getenv('PROJ_LIB'))
print(os.getcwd())

Current Working Directory 
*** C:\Program Files\PostgreSQL\11\gdal-data
is dir: True
is file: True
is readable: True
C:\\Users\\00073294\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\pyproj\\proj_dir\\share\\proj
C:\Users\00073294\Dropbox\1_Jupyter_notebooks\map2loop\notebooks


## Create bounding box based on inputs

In [11]:
test_data_path='../test_data3/'


%run -i "../test_data3/m2l_config.py"

bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
lat_point_list = [miny, miny, maxy, maxy, maxy]
lon_point_list = [minx, maxx, maxx, minx, minx]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom]) 
bbox=(minx,miny,maxx,maxy)


Default parameters loaded from m2l_config.py:
#ROI
minlong=117 # should back calc from metre system\n",
maxlong=118
minlat=-23
maxlat=-22

step_out=0
inset=0

minx=500057
maxx=603028
miny=7455348
maxy=7567953

#CRS
src_crs = {'init': 'EPSG:4326'}  
dst_crs = {'init': 'EPSG:28350'} 

#PATH
mname='hams3'
geology_file='hams2_geol.shp'
fault_file='GEOS_GEOLOGY_LINEARSTRUCTURE_500K_GSD.shp'
structure_file='hams2_structure.shp'

test_data_path='../test_data3/'


#CODES
gcode='GROUP_'
dcode='DIP'
ddcode='DIP_DIR'
ccode='CODE'
ncode='NAME'
ocode='OBJECTID'
gicode='GEOPNT_ID'
r1code='ROCKTYPE1'
r2code='ROCKTYPE2'
tcode='TYPE'
fcode='FEATURE'
dscode='DESCRIPTN'
ucode='UNITNAME'
mincode='MIN_AGE_MA'
maxcode='MAX_AGE_MA'
sill_label='sill'
intrusive_label='intrusive'
fold_label='Fold axial trace'
fault_label='Fault'
bedding_label='Bed'

#MODEL_EXTENTS
model_base=-8200


#DECIMATION
orientation_decimate=0
contact_decimate=25
fault_decimate=5
fold_decimate=5


#INTERPOLATION
gridx=100
gridy=100
schem

## First we test to see if we have access to the online data we need


In [ ]:
net=m2l_utils.have_access("geo.loop-gis.org")


## Optional WFS source
  
WFS brings in field names as lower case, so need to redefine codes too

In [ ]:
#local_paths=True
if(not local_paths and net):
    structure_file='http://geo.loop-gis.org/geoserver/loop/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=warox_points_f5011&bbox='+bbox2+'&srs=EPSG:28350'
    fault_file='http://geo.loop-gis.org/geoserver/loop/wfs?service=WFS&version=1.1.0&request=GetFeature&typeName=linear_500k&bbox='+bbox2+'&srs=EPSG:28350'
    geology_file='http://geo.loop-gis.org/geoserver/loop/wfs?service=WFS&version=1.0.0&request=GetFeature&typeName=loop:geol_500k&bbox='+bbox2+'&srs=EPSG:28350'

    gcode='group_'
    dcode='dip'
    ddcode='dip_dir'
    ccode='code'
    ncode='fname'
    ocode='objectid'
    gicode='geopnt_id'
    r1code='rocktype1'
    r2code='rocktype2'
    tcode='type'
    fcode='feature'
    dscode='descriptn'
    ucode='unitname'
    mincode='min_age_ma'
    maxcode='max_age_ma'

## Plot geology polygons and bounding box

In [ ]:
geology_ll = gpd.read_file(geology_file,bbox=bbox)

base=geology_ll.plot(column=ccode,figsize=(10,10),edgecolor='#000000',linewidth=0.2)
polygon.plot(ax=base, color='none',edgecolor='black')


## Save to file as WKT

In [ ]:
sub_geol = geology_ll[['geometry', ocode,ccode,gcode,ucode,mincode,maxcode,dscode,r1code,r2code]]

In [ ]:
f= open(geology_file_csv,"w+")
f.write("WKT\tobjectid\tunitname\tgroup_\tmin_age_ma\tmax_age_ma\tcode\trocktype1\trocktype2\tdescriptn\n")
#display(sub_geol)        
print(len(sub_geol)," polygons")
#print(sub_geol)
for i in range(0,len(sub_geol)):
    f.write("\""+str(sub_geol.loc[i].geometry)+"\"\t")
    f.write("\""+str(sub_geol.loc[i][ocode])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][ccode])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][gcode]).replace("None","")+"\"\t") #since map2model is lookig for "" not "None"
    f.write("\""+str(sub_geol.loc[i][mincode])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][maxcode])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][ucode])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][r1code])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][r2code])+"\"\t")
    f.write("\""+str(sub_geol.loc[i][dscode])+"\"\n")
    
f.close()

## Read and save WAROX point data as WKT

In [ ]:
warox = gpd.read_file(structure_file,bbox=bbox)


sub_pts = warox[['geometry', gicode,dcode,ddcode]]

f= open(structure_file_csv,"w+")
f.write("WKT\tgeopnt_id\tdip\tdip_dir\n")

print(len(sub_pts)," points")


#for i in range(0,len(sub_pts)):
#    for j in range(0,len(sub_geol)):
#        if(sub_pts.loc[i].geometry.within(sub_geol.loc[j].geometry)):
#            print(i,j)

for i in range(0,len(sub_pts)):
    line="\""+str(sub_pts.loc[i].geometry)+"\"\t\""+str(sub_pts.loc[i][gicode])+"\"\t\""+\
      str(sub_pts.loc[i][dcode])+"\"\t\""+str(sub_pts.loc[i][ddcode])+"\"\n"    
    f.write(functools.reduce(operator.add, (line)))
    
f.close()

base=sub_pts.plot()
polygon.plot(ax=base, color='none',edgecolor='black')


## Plot faults and bounding box

In [ ]:
#bbox='-23,117,-22,118'
#lines=gpd.read_file("https://services.slip.wa.gov.au/public/services/SLIP_Public_Services/Geology_and_Soils_Map_WFS/MapServer/WFSServer/?service=WFS&version=1.0.0&request=GetFeature&typename=SLIP_Public_Services_Geology_and_Soils_Map_WFS:__2_500_000_State_interpreted_bedrock_geology_structural_lines__DMIRS-013_&BBOX="+bbox+"&srs=EPSG:3857")
#print(lines)
#lines.plot(column='feature',figsize=(10,10),linewidth=0.2)
lines_ll=gpd.read_file(fault_file,bbox=bbox)
#lines_ll = lines.to_crs('epsg:4326')
#print(geology.crs)
base2=lines_ll.plot(cmap='rainbow',column=fcode,figsize=(10,10),linewidth=0.4)
polygon.plot(ax=base2, color='none',edgecolor='black')


## Save faults to file as WKT

In [ ]:
sub_lines = lines_ll[['geometry', ocode,fcode]]

f= open(fault_file_csv,"w+")
f.write("WKT\tobjectid\tfeature\n")

print(len(sub_lines)," polylines")

for i in range(0,len(sub_lines)):
    if('Fault' in sub_lines.loc[i][fcode]):
        f.write("\""+str(sub_lines.loc[i].geometry)+"\"\t")
        f.write("\""+str(sub_lines.loc[i][ocode])+"\"\t")
        f.write("\""+str(sub_lines.loc[i][fcode])+"\"\n")
    
f.close()

## Create map2model input file

In [ ]:
f=open(m2m_cpp_path+'Parfile','w')
f.write('--- COLUMN NAMES IN CSV DATA FILES: -------------------------------------------------------------\n')
f.write('OBJECT COORDINATES              =WKT\n')
f.write('FAULT: ID                       ='+ocode+'\n')
f.write('FAULT: FEATURE                  ='+fcode+'\n')
f.write('POINT: ID                       ='+gicode+'\n')
f.write('POINT: DIP                      ='+dcode+'\n')
f.write('POINT: DIP DIR                  ='+ddcode+'\n')
f.write('POLYGON: ID                     ='+ocode+'\n')
f.write('POLYGON: LEVEL1 NAME            ='+ucode+'\n')
f.write('POLYGON: LEVEL2 NAME            ='+gcode+'\n')
f.write('POLYGON: MIN AGE                ='+mincode+'\n')
f.write('POLYGON: MAX AGE                ='+maxcode+'\n')
f.write('POLYGON: CODE                   ='+ccode+'\n')
f.write('POLYGON: DESCRIPTION            ='+dscode+'\n')
f.write('POLYGON: ROCKTYPE1              ='+r1code+'\n')
f.write('POLYGON: ROCKTYPE2              ='+r2code+'\n')
f.write('--- SOME CONSTANTS: ----------------------------------------------------------------------------\n')
f.write('FAULT AXIAL FEATURE NAME        ='+fold_label+'\n')
f.write('SILL UNIT DESCRIPTION CONTAINS  ='+sill_label+'\n')
f.write('------------------------------------------------------------------------------------------------\n')
f.write('Path to the output data folder                      ='+graph_path+'\n')
f.write('Path to geology data file                           ='+geology_file_csv+'\n')
f.write('Path to faults data file                            ='+fault_file_csv+'\n')
f.write('Path to points data file                            ='+structure_file_csv+'\n')
f.write('------------------------------------------------------------------------------------------------\n')
f.write('Clipping window X1 Y1 X2 Y2 (zeros for infinite)    ='+str(minx)+' '+str(miny)+' '+str(maxx)+' '+str(maxy)+'\n')
f.write('Min length fraction for strat/fault graphs          =0.0\n')
f.write('Graph edge width categories (three doubles)         =2000. 20000. 200000.\n')
f.write('Graph edge direction (0-min age, 1-max age, 2-avg)  =2\n')
f.write('Partial graph polygon ID                            =32\n')
f.write('Partial graph depth                                 =4\n')
f.write('Map subregion size dx, dy [m] (zeros for full map)  =0. 0.\n')
f.write('------------------------------------------------------------------------------------------------\n')
f.close()



## Calculate topology

In [ ]:
os.chdir(m2m_cpp_path)
print(os.getcwd())
subprocess.run(["map2model.exe", "Parfile"],shell = True)

## Simple network graph of the geology with legend

In [ ]:
G=nx.read_gml(strat_graph_file,label='id')
selected_nodes = [n for n,v in G.nodes(data=True) if n >=0]
nx.draw_networkx(G, pos=nx.kamada_kawai_layout(G), arrows=True, nodelist=selected_nodes)

nlist=list(G.nodes.data('LabelGraphics'))
nlist.sort()
for no in nlist:
    if(no[0]>=0):
        elem=str(no[1]).replace("{'text':","").replace(", 'fontSize': 14}","")
        #second=elem.split(":").replace("'","")
        print(no[0]," ",elem)

## Detailed Topology via dot or gml format files
For a more detailed look we can open up the online version of <b>yEd</b> <a href="https://www.yworks.com/yed-live/" >https://www.yworks.com/yed-live/</a>  and load in the *.dot file that we will now create or dowload the <a href="https://www.yworks.com/products/yed/download">yEd program</a> and look at the *.gml files in the <b>output</b> directory

In [ ]:
from networkx.drawing.nx_pydot import write_dot

import filer,gfiler

filepath=filer.gui_fname().decode('UTF-8')
write_dot(G,filepath)

http://localhost:8888/notebooks/Dropbox/1_Jupyter_notebooks/map2loop/notebooks/2.%20map2loop_after_topology.ipynb